In [1]:
import numpy as np
import torch
import copy

In [71]:
def apply_species_collision(species1, species2, complex_, collision_rate, time_step):
    
    collision_effect = collision_rate * time_step
    complex_formed = (np.minimum(species1.T * collision_effect, species2.T * collision_effect)).T
    complex_formed = np.maximum(complex_formed, 0)

    updated_species1 = np.maximum(species1 - complex_formed, 0)
    updated_species2 = np.maximum(species2 - complex_formed, 0)

    updated_complex = complex_ + complex_formed

    return updated_species1, updated_species2, updated_complex

In [72]:
g = np.ones((5, 7, 10, 10)) # (m, z, y, x)

In [73]:
g[:, 0, :, 1] = 30
g[:, 1, :, 1] = 50
g[:, 2, :, 1] = 100
g[:, -1, -1, -1] = np.array([0.5,.4,.6,.8,.10])

In [74]:
s1, s2, x = apply_species_collision(g[:, 0, :, 1], g[:, 1, :, 1], g[:, 2, :, 1], g[:, -1, -1, -1], 0.2)

In [75]:
s1

array([[27. , 27. , 27. , 27. , 27. , 27. , 27. , 27. , 27. , 27. ],
       [27.6, 27.6, 27.6, 27.6, 27.6, 27.6, 27.6, 27.6, 27.6, 27.6],
       [26.4, 26.4, 26.4, 26.4, 26.4, 26.4, 26.4, 26.4, 26.4, 26.4],
       [25.2, 25.2, 25.2, 25.2, 25.2, 25.2, 25.2, 25.2, 25.2, 25.2],
       [29.4, 29.4, 29.4, 29.4, 29.4, 29.4, 29.4, 29.4, 29.4, 29.4]])

In [76]:
s2

array([[47. , 47. , 47. , 47. , 47. , 47. , 47. , 47. , 47. , 47. ],
       [47.6, 47.6, 47.6, 47.6, 47.6, 47.6, 47.6, 47.6, 47.6, 47.6],
       [46.4, 46.4, 46.4, 46.4, 46.4, 46.4, 46.4, 46.4, 46.4, 46.4],
       [45.2, 45.2, 45.2, 45.2, 45.2, 45.2, 45.2, 45.2, 45.2, 45.2],
       [49.4, 49.4, 49.4, 49.4, 49.4, 49.4, 49.4, 49.4, 49.4, 49.4]])

In [77]:
x

array([[103. , 103. , 103. , 103. , 103. , 103. , 103. , 103. , 103. ,
        103. ],
       [102.4, 102.4, 102.4, 102.4, 102.4, 102.4, 102.4, 102.4, 102.4,
        102.4],
       [103.6, 103.6, 103.6, 103.6, 103.6, 103.6, 103.6, 103.6, 103.6,
        103.6],
       [104.8, 104.8, 104.8, 104.8, 104.8, 104.8, 104.8, 104.8, 104.8,
        104.8],
       [100.6, 100.6, 100.6, 100.6, 100.6, 100.6, 100.6, 100.6, 100.6,
        100.6]])

In [115]:
def update_lower_right_corner_concentration(
    cell_concentration,
    upper_cell_concentration,
    left_cell_concentration,
    diffusion_rate,
    time_step):
   
    in_diffusion = (time_step * upper_cell_concentration * diffusion_rate) + \
                   (time_step * left_cell_concentration * diffusion_rate)
    out_diffusion = time_step * cell_concentration * diffusion_rate * 2

    updated_concentration = cell_concentration + in_diffusion - out_diffusion

    return updated_concentration


In [116]:
g = np.ones((5, 7, 10, 10)) # (m, z, y, x)

In [117]:
g[:, 0, -1, -1] = 10
g[:, 0, -2, -1] = 20
g[:, 0, -1, -2] = 30


In [118]:
g[:, 0, -1, -1] = update_lower_right_corner_concentration(g[:, 0, -1, -1], g[:, 0, -2, -1], g[:, 0, -1, -2], np.array([2, 3, 1, 5, 1.2]), .2)

In [121]:
g[:, 0, -1, -1]

array([22. , 28. , 16. , 40. , 17.2])

In [128]:
def update_upper_left_corner_concentration(
    cell_concentration,
    lower_cell_concentration,
    right_cell_concentration,
    diffusion_rate,
    time_step):
  
    in_diffusion = (time_step * lower_cell_concentration * diffusion_rate) + \
                   (time_step * right_cell_concentration * diffusion_rate)
    out_diffusion = time_step * cell_concentration * diffusion_rate * 2

    updated_concentration = cell_concentration + in_diffusion - out_diffusion

    return updated_concentration

In [129]:
g = np.ones((5, 7, 10, 10)) # (m, z, y, x)

In [130]:
g[:, 0, 0, 0] = 10
g[:, 0, 1, 0] = 20
g[:, 0, 0, 1] = 30

In [131]:
g[:, 0, 0, 0] = update_upper_left_corner_concentration(g[:, 0, 0, 0], g[:, 0, 1, 0], g[:, 0, 0, 1], np.array([2, 3, 1, 5, 1.2]), .2)

In [135]:
def update_upper_right_corner_concentration(
    cell_concentration,
    lower_cell_concentration,
    left_cell_concentration,
    diffusion_rate,
    time_step):
  
    in_diffusion = (time_step * lower_cell_concentration * diffusion_rate) + \
                   (time_step * left_cell_concentration * diffusion_rate)
    out_diffusion = time_step * cell_concentration * diffusion_rate * 2

    updated_concentration = cell_concentration + in_diffusion - out_diffusion

    return updated_concentration

In [136]:
g = np.ones((5, 7, 10, 10)) # (m, z, y, x)

In [137]:
g[:, 0, 0, -1] = 10
g[:, 0, 1, -1] = 20
g[:, 0, 0, -2] = 30

In [139]:
g[:, 0, 0, -1] = update_upper_right_corner_concentration(g[:, 0, 0, -1], g[:, 0, 1, -1], g[:, 0, 0, -2], np.array([2, 3, 1, 5, 1.2]), .2)

In [155]:
def update_left_side_concentration(
    cell_concentration,
    upper_cell_concentration,
    lower_cell_concentration,
    right_cell_concentration,
    diffusion_rate,
    time_step):
    
    upper_cell_in = (time_step * upper_cell_concentration.T * diffusion_rate).T
    lower_cell_in = (time_step * lower_cell_concentration.T * diffusion_rate).T
    right_cell_in = (time_step * right_cell_concentration.T * diffusion_rate).T
    in_diffusion = upper_cell_in + lower_cell_in + right_cell_in

    out_diffusion = (time_step * cell_concentration.T * diffusion_rate * 3).T

    updated_concentration = cell_concentration + in_diffusion - out_diffusion

    return updated_concentration

In [156]:
g = np.ones((5, 7, 10, 10)) # (m, z, y, x)

In [157]:
g[:, 0, :, 0] = 1.9
g[:, 0, :, 1] = 2.7

In [158]:
g[:, 0, 1:-1, 0] = update_left_side_concentration(g[:, 0, 1:-1, 0], g[:, 0, :-2, 0] , g[:, 0, 2: , 0] , g[:, 0, 1:-1, 1], np.array([1,2,3,4,5]), 0.4)

In [160]:
g[:, 0, 1:-1, 0].shape

(5, 8)

In [165]:
def update_right_side_concentration(
        cell_concentration,
        upper_cell_concentration,
        lower_cell_concentration,
        left_cell_concentration,
        diffusion_rate,
        time_step):
  
    upper_cell_in = (time_step * upper_cell_concentration.T * diffusion_rate).T
    lower_cell_in = (time_step * lower_cell_concentration.T * diffusion_rate).T
    right_cell_in = (time_step * left_cell_concentration.T * diffusion_rate).T
    in_diffusion = upper_cell_in + lower_cell_in + right_cell_in

    out_diffusion = (time_step * cell_concentration.T * diffusion_rate * 3).T

    updated_concentration = cell_concentration + in_diffusion - out_diffusion

    return updated_concentration

In [166]:
g = np.ones((5, 7, 10, 10)) # (m, z, y, x)

In [167]:
g[:, 0, :, -1] = 1.9
g[:, 0, :, -2] = 2.7

In [168]:
g[:, 0, 1:-1, -1] = update_right_side_concentration(g[:, 0, 1:-1, -1], g[:, 0, :-2, -1] , g[:, 0, 2: , -1] , g[:, 0, 1:-1, -2], np.array([1,2,3,4,5]), 0.4)

In [200]:
def update_central_concentration_upper(
        cell_concentration,
        lower_cell_concentration,
        right_cell_concentration,
        left_cell_concentration,
        diffusion_rate,
        time_step):

    lower_cell_in = time_step * lower_cell_concentration * diffusion_rate
    right_cell_in = time_step * right_cell_concentration * diffusion_rate
    left_cell_in = time_step * left_cell_concentration * diffusion_rate

    in_diffusion = lower_cell_in + right_cell_in + left_cell_in
    out_diffusion = time_step * cell_concentration * diffusion_rate * 3

    updated_concentration = cell_concentration + in_diffusion - out_diffusion

    return updated_concentration

In [201]:
g = np.ones((5, 7, 10, 10)) # (m, z, y, x)

In [202]:
g[:, 0, :, 2] = 5
g[:, 0, 1, 3] = 15
g[:, 0, 1, 3] = 67
g[:, 0, :, 4] = 25

In [203]:
g[:, 0, 0, 3] = update_central_concentration_upper(g[:, 0, 0, 3], g[:, 0, 1, 3], g[:, 0, 0, 4], g[:, 0, 0, 2], np.array([1,2,3,4,5]), 0.3)

In [207]:
def update_central_concentration_lower(
        cell_concentration,
        upper_cell_concentration,
        right_cell_concentration,
        left_cell_concentration,
        diffusion_rate,
        time_step):
  
    upper_cell_in = time_step * upper_cell_concentration * diffusion_rate
    right_cell_in = time_step * right_cell_concentration * diffusion_rate
    left_cell_in = time_step * left_cell_concentration * diffusion_rate

    in_diffusion = upper_cell_in + right_cell_in + left_cell_in
    out_diffusion = time_step * cell_concentration * diffusion_rate * 3

    updated_concentration = cell_concentration + in_diffusion - out_diffusion

    return updated_concentration

In [208]:
g = np.ones((5, 7, 10, 10)) # (m, z, y, x)

In [209]:
g[:, 0, :, 2] = 5
g[:, 0, :, 3] = 15
g[:, 0, -2, 3] = 67
g[:, 0, :, 4] = 25

In [210]:
g[0, 0, :, :]

array([[ 1.,  1.,  5., 15., 25.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  5., 15., 25.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  5., 15., 25.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  5., 15., 25.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  5., 15., 25.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  5., 15., 25.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  5., 15., 25.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  5., 15., 25.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  5., 67., 25.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  5., 15., 25.,  1.,  1.,  1.,  1.,  1.]])

In [211]:
g[:, 0, -1, 3] = update_central_concentration_lower(g[:, 0, -1, 3], g[:, 0, -2, 3], g[:, 0, -1, 4], g[:, 0, -1, 2], np.array([1,2,3,4,5]), 0.3)

In [212]:
g[0, 0, :, :]

array([[ 1. ,  1. ,  5. , 15. , 25. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 1. ,  1. ,  5. , 15. , 25. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 1. ,  1. ,  5. , 15. , 25. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 1. ,  1. ,  5. , 15. , 25. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 1. ,  1. ,  5. , 15. , 25. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 1. ,  1. ,  5. , 15. , 25. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 1. ,  1. ,  5. , 15. , 25. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 1. ,  1. ,  5. , 15. , 25. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 1. ,  1. ,  5. , 67. , 25. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 1. ,  1. ,  5. , 30.6, 25. ,  1. ,  1. ,  1. ,  1. ,  1. ]])

In [232]:
def update_central_concentration_middle(
        cell_concentration,
        upper_cell_concentration,
        lower_cell_concentration,
        right_cell_concentration,
        left_cell_concentration,
        diffusion_rate,
        time_step):

    upper_cell_in = (time_step * upper_cell_concentration.T * diffusion_rate).T
    lower_cell_in = (time_step * lower_cell_concentration.T * diffusion_rate).T
    right_cell_in = (time_step * right_cell_concentration.T * diffusion_rate).T
    left_cell_in = (time_step * left_cell_concentration.T * diffusion_rate).T

    in_diffusion = upper_cell_in + lower_cell_in + right_cell_in + left_cell_in
    out_diffusion = (time_step * cell_concentration.T * diffusion_rate).T * 4

    updated_concentration = cell_concentration + in_diffusion - out_diffusion

    return updated_concentration

In [233]:
g = np.ones((5, 7, 10, 10)) # (m, z, y, x)

In [234]:
g[:, 0, :, 2] = 5
g[:, 0, :, 3] = 15
g[:, 0, -5, 3] = 67
g[:, 0, :, 4] = 25

In [235]:
g[0, 0, :, :]

array([[ 1.,  1.,  5., 15., 25.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  5., 15., 25.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  5., 15., 25.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  5., 15., 25.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  5., 15., 25.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  5., 67., 25.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  5., 15., 25.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  5., 15., 25.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  5., 15., 25.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  5., 15., 25.,  1.,  1.,  1.,  1.,  1.]])

In [230]:
g[:, 0, -5, 3] = update_central_concentration_middle(g[:, 0, -5, 3], g[:, 0, -6, 3], g[:, 0, -4, 3], g[:, 0, -5, 4], g[:, 0, -5, 2], np.array([1,2,3,4,5]), 0.3)

In [231]:
g[0, 0, :, :]

array([[ 1. ,  1. ,  5. , 15. , 25. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 1. ,  1. ,  5. , 15. , 25. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 1. ,  1. ,  5. , 15. , 25. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 1. ,  1. ,  5. , 15. , 25. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 1. ,  1. ,  5. , 15. , 25. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 1. ,  1. ,  5. ,  4.6, 25. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 1. ,  1. ,  5. , 15. , 25. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 1. ,  1. ,  5. , 15. , 25. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 1. ,  1. ,  5. , 15. , 25. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 1. ,  1. ,  5. , 15. , 25. ,  1. ,  1. ,  1. ,  1. ,  1. ]])

In [244]:
g = np.ones((10, 20, 30))
g[:, :, 0].shape

(10, 20)

In [245]:
v = g[:, :, 0]

In [246]:
m = v[:, 0]

In [ ]:
"""
Compartment size times:  [10.832071781158447, 0.11558675765991211, 0.36243414878845215, 2.0385539531707764, 16.384703159332275, 107.51107883453369]
Simulation time for different epochs and time steps:  [0.0251920223236084, 0.11624932289123535, 0.2307727336883545, 2.3095428943634033, 11.656088590621948, 23.539443016052246]
"""

In [115]:
import time

def population_simulation(individual):
    """
    Simulate the dynamics of a population of individuals within a specified compartmental system.

    Parameters:
    - population (np.ndarray or list):
        - If a numpy array, it should have a shape of (m, z, y, x), where:
            - m: number of individuals
            - z: number of species in the system
            - y: number of compartment rows
            - x: number of compartment columns
        - If a list, it represents a list of individual populations (to be implemented).

    - sim_params (bool, optional):
        - If False, all individuals use the same simulation parameters.
        - If True , each individual has specific simulation parameters.

    Returns:
    - np.ndarray: An array of shape (m, y, x) containing the final results for each individual,
      where m is the number of individuals, y and x are the compartment shape.
    """

    z, y, x = individual.shape  # z: species, (y, x): compartment's shape
    num_iters = int(x)  # Number of iterations in each epoch (equal to x)
    # it is equal to x because system will iterate from left to right of the compartment in each epoch
    num_species = int(individual[-1, -1, 0])  # number os species present in the system
    num_pairs = int(individual[-1, -1, 1])  # number of pairs present in the system
    max_epoch = int(individual[-1, -1, 2])  # maximum number of epochs
    stop = int(individual[-1, -1, 3])  # simulation duration
    time_step = individual[-1, -1, 4]  # time step
    num_epochs = int(stop / time_step)  # Total number of epochs
    pair_start = int(num_species * 2)  # Starting index for species pairs
    pair_stop = int(pair_start + (num_pairs * 2))  # Ending index for species pairs

    epoch = 0
    while epoch < max_epoch or epoch < num_epochs:

        for i in range(num_iters):

            # Update species production
            for j in range(0, num_species*2, 2):
                individual[j, :, i] = apply_component_production(
                    initial_concentration=individual[j, :, i],
                    production_pattern=individual[j+1, :, i],
                    production_rate=individual[-1, j, 0],
                    time_step=time_step
                )

            # Handle species collision
            for j in range(pair_start, pair_stop, 2):
                (individual[int(individual[j+1, 0, 0]), :, i],
                 individual[int(individual[j+1, 0, 1]), :, i],
                 individual[j, :, i]) = apply_species_collision(
                    species1=individual[int(individual[j+1, 0, 0]), :, i],
                    species2=individual[int(individual[j+1, 0, 1]), :, i],
                    complex_=individual[j, :, i],
                    collision_rate=individual[j+1, 1, 0],
                    time_step=time_step
                )

            # Update species degradation
            for j in range(0, num_species*2, 2):
                individual[j, :, i] = apply_component_degradation(
                    initial_concentration=individual[j, :, i],
                    degradation_rate=individual[-1, j, 1],
                    time_step=time_step
                )

            # Handle complex degradation
            for j in range(pair_start, num_pairs, 2):
                individual[j, :, i] = apply_component_degradation(
                    initial_concentration=individual[j, :, i],
                    degradation_rate=individual[j+1, 1, 2],
                    time_step=time_step
                )

            # Handle complex dissociation
            for j in range(pair_start, pair_stop, 2):
                (individual[int(individual[j+1, 0, 0]), :, i],
                 individual[int(individual[j+1, 0, 1]), :, i],
                 individual[j, :, i]) = apply_complex_dissociation(
                    species1=individual[int(individual[j+1, 0, 0]), :, i],
                    species2=individual[int(individual[j+1, 0, 1]), :, i],
                    complex_=individual[j, :, i],
                    dissociation_rate=individual[j+1, 1, 1],
                    time_step=time_step
                )

            # Update species diffusion
            for j in range(0, num_species*2, 2):
                individual[j, :, i] = apply_diffusion(
                    current_concentration=individual[j, :, i],
                    compartment=individual[j, :, :],
                    column_position=i,
                    diffusion_rate=individual[-1, j, 2],
                    time_step=time_step
                )

            # Handle complex diffusion
            for j in range(pair_start, num_pairs, 2):
                individual[j, :, i] = apply_diffusion(
                    current_concentration=individual[j, :, i],
                    compartment=individual[j, :, :],
                    column_position=i,
                    diffusion_rate=individual[j+1, 1, 3],
                    time_step=time_step
                )

        epoch += 1

    return individual[0, :, :]

def apply_diffusion(current_concentration, compartment, column_position, diffusion_rate, time_step):
    """
    Apply diffusion to update the concentration of species in a specific column of a 2D compartment for all individual in population.

    Parameters:
    - current_concentration (2d array): Array of current concentrations for each cell.
    - compartment (3d array): Array representing the 2D compartment where diffusion takes place for all individual in population.
    - column_position (int): Column position of the cells being updated (0-based index).
    - diffusion_rates (1d array): Rates at which the species diffuses between cells.
    - time_step (float/1d array): Discrete time step/s for the calculation.

    Returns:
    - 2d array: updated current_concentration array.
    """
    compartment_size = compartment.shape[1]
    temporary_concentration = np.copy(current_concentration)

    if column_position == 0:

        # Update concentration for the upper-left corner cell
        temporary_concentration[0] = update_upper_left_corner_concentration(
            cell_concentration=current_concentration[0],
            lower_cell_concentration=compartment[1, 0],
            right_cell_concentration=compartment[0, 1],
            diffusion_rate=diffusion_rate,
            time_step=time_step
        )

        # Update concentration for the lower-left corner cell
        temporary_concentration[-1] = update_lower_left_corner_concentration(
            cell_concentration=current_concentration[-1],
            upper_cell_concentration=compartment[-2, 0],
            right_cell_concentration=compartment[-1, 1],
            diffusion_rate=diffusion_rate,
            time_step=time_step
        )

        # Update concentrations for the left-side cells (excluding corners)
        temporary_concentration[1:-1] = update_left_side_concentration(
            cell_concentration=current_concentration[1:-1],
            upper_cell_concentration=compartment[:-2, 0],
            lower_cell_concentration=compartment[2:, 0],
            right_cell_concentration=compartment[1:-1, 1],
            diffusion_rate=diffusion_rate,
            time_step=time_step
        )

    elif column_position == compartment_size - 1:

        # Update concentration for the upper-right corner cell
        temporary_concentration[0] = update_upper_right_corner_concentration(
            cell_concentration=current_concentration[0],
            lower_cell_concentration=compartment[1, -1],
            left_cell_concentration=compartment[0, -2],
            diffusion_rate=diffusion_rate,
            time_step=time_step
        )

        # Update concentration for the lower-right corner cell
        temporary_concentration[-1] = update_lower_right_corner_concentration(
            cell_concentration=current_concentration[-1],
            upper_cell_concentration=compartment[-2, -1],
            left_cell_concentration=compartment[-1, -2],
            diffusion_rate=diffusion_rate,
            time_step=time_step
        )

        # Update concentrations for the left-side cells (excluding corners)
        temporary_concentration[1:-1] = update_right_side_concentration(
            cell_concentration=current_concentration[1:-1],
            upper_cell_concentration=compartment[0:-2, -1],
            lower_cell_concentration=compartment[2:, -1],
            left_cell_concentration=compartment[1:-1, -2],
            diffusion_rate=diffusion_rate,
            time_step=time_step
        )


    else:

        temporary_concentration[0] = update_central_concentration_upper(
            cell_concentration=current_concentration[0],
            lower_cell_concentration=compartment[1, column_position],
            right_cell_concentration=compartment[0, column_position + 1],
            left_cell_concentration=compartment[0, column_position - 1],
            diffusion_rate=diffusion_rate,
            time_step=time_step)

        temporary_concentration[-1] = update_central_concentration_lower(
            cell_concentration=current_concentration[-1],
            upper_cell_concentration=compartment[-2, column_position],
            right_cell_concentration=compartment[-1, column_position + 1],
            left_cell_concentration=compartment[-1, column_position - 1],
            diffusion_rate=diffusion_rate,
            time_step=time_step)

        temporary_concentration[1:-1] = update_central_concentration_middle(
            cell_concentration=current_concentration[1:-1],
            upper_cell_concentration=compartment[0:-2, column_position],
            lower_cell_concentration=compartment[2:, column_position],
            right_cell_concentration=compartment[1:-1, column_position+1],
            left_cell_concentration=compartment[1:-1, column_position-1],
            diffusion_rate=diffusion_rate,
            time_step=time_step
        )

    updated_concentration = np.maximum(temporary_concentration, 0)

    return updated_concentration




def update_lower_left_corner_concentration(
    cell_concentration,
    upper_cell_concentration,
    right_cell_concentration,
    diffusion_rate,
    time_step):
    """
    Update the concentration of a species in a cell located at the lower-left corner of a 2D compartment
    based on diffusion from neighboring cells.

    Parameters:
    - cell_concentration (1d array): Concentration of the species in the lower-left corner cell.
    - upper_cell_concentration (1d array): Concentration of the species in the cell directly above the lower-left cell.
    - right_cell_concentration (1d array): Concentration of the species in the cell directly to the right of the lower-left cell.
    - diffusion_rate (float): Rate at which the species diffuses between cells.
    - time_step (float): Discrete time step for the calculation.

    Returns:
    - 1d array: Updated concentration of the species in the lower-left corner cell.
    """
    in_diffusion = (time_step * upper_cell_concentration * diffusion_rate) + \
                   (time_step * right_cell_concentration * diffusion_rate)
    out_diffusion = time_step * cell_concentration * diffusion_rate * 2

    updated_concentration = cell_concentration + in_diffusion - out_diffusion

    return updated_concentration


def update_lower_right_corner_concentration(
    cell_concentration,
    upper_cell_concentration,
    left_cell_concentration,
    diffusion_rate,
    time_step):
    """
        Update the concentration of a species in a cell located at the lower-right corner of a 2D compartment
        based on diffusion from neighboring cells.

        Parameters:
        - cell_concentration (1d array): Concentration of the species in the lower-left corner cell.
        - upper_cell_concentration (1d array): Concentration of the species in the cell directly above the lower-right cell.
        - left_cell_concentration (1d array): Concentration of the species in the cell directly to the left of the lower-right cell.
        - diffusion_rate (float): Rate at which the species diffuses between cells.
        - time_step (float): Discrete time step for the calculation.

        Returns:
        - 1d array: Updated concentration of the species in the lower-left corner cell.
        """
    in_diffusion = (time_step * upper_cell_concentration * diffusion_rate) + \
                   (time_step * left_cell_concentration * diffusion_rate)
    out_diffusion = time_step * cell_concentration * diffusion_rate * 2

    updated_concentration = cell_concentration + in_diffusion - out_diffusion

    return updated_concentration



def update_upper_left_corner_concentration(
    cell_concentration,
    lower_cell_concentration,
    right_cell_concentration,
    diffusion_rate,
    time_step):
    """
    Update the concentration of a species in a cell located at the upper-left corner of a 2D compartment
    based on diffusion from neighboring cells for each individual in population.

    Parameters:
    - cell_concentration (1d array): Concentration of the species in the upper-left corner cell.
    - lower_cell_concentration (1d array): Concentration of the species in the cell directly below the upper-left cell.
    - right_cell_concentration (1d array): Concentration of the species in the cell directly to the right of the upper-left cell.
    - diffusion_rate (float): Rates at which the species diffuses between cells.
    - time_step (float): Discrete time step for the calculation.

    Returns:
    - 1d array: Updated concentration of the species in the upper-left corner cell.
    """
    in_diffusion = (time_step * lower_cell_concentration * diffusion_rate) + \
                   (time_step * right_cell_concentration * diffusion_rate)
    out_diffusion = time_step * cell_concentration * diffusion_rate * 2

    updated_concentration = cell_concentration + in_diffusion - out_diffusion

    return updated_concentration


def update_upper_right_corner_concentration(
    cell_concentration,
    lower_cell_concentration,
    left_cell_concentration,
    diffusion_rate,
    time_step):
    """
        Update the concentration of a species in a cell located at the upper-right corner of a 2D compartment
        based on diffusion from neighboring cells.

        Parameters:
        - cell_concentration (1d array): Concentration of the species in the upper-right corner cell.
        - lower_cell_concentration (1d array): Concentration of the species in the cell directly below the upper-left cell.
        - left_cell_concentration (1d array): Concentration of the species in the cell directly to the left of the upper-right cell.
        - diffusion_rate (float): Rate at which the species diffuses between cells.
        - time_step (float/1d array): Discrete time step/s for the calculation.

        Returns:
        - 1d array: Updated concentration of the species in the upper-left corner cell.
        """
    in_diffusion = (time_step * lower_cell_concentration * diffusion_rate) + \
                   (time_step * left_cell_concentration * diffusion_rate)
    out_diffusion = time_step * cell_concentration * diffusion_rate * 2

    updated_concentration = cell_concentration + in_diffusion - out_diffusion

    return updated_concentration



def update_left_side_concentration(
    cell_concentration,
    upper_cell_concentration,
    lower_cell_concentration,
    right_cell_concentration,
    diffusion_rate,
    time_step):
    """
    Update the concentration of a species in cells located along the left side of a 2D compartment
    (excluding the corners) based on diffusion from neighboring cells for each individual in population.

    Parameters:
    - cell_concentration (2d array): Array of concentrations for cells in the leftmost column (excluding corners).
    - upper_cell_concentration (2d array): Array of concentrations for cells directly above the current cells.
    - lower_cell_concentration (2d array): Array of concentrations for cells directly below the current cells.
    - right_cell_concentration (2d array): Array of concentrations for cells directly to the right of the current cells.
    - diffusion_rate (float): Rate at which the species diffuses between cells.
    - time_step (float): Discrete time step for the calculation.

    Returns:
    - numpy.ndarray: Updated concentrations of the species in the current cells.
    """
    upper_cell_in = time_step * upper_cell_concentration * diffusion_rate
    lower_cell_in = time_step * lower_cell_concentration * diffusion_rate
    right_cell_in = time_step * right_cell_concentration * diffusion_rate

    in_diffusion = upper_cell_in + lower_cell_in + right_cell_in
    out_diffusion = time_step * cell_concentration * diffusion_rate * 3

    updated_concentration = cell_concentration + in_diffusion - out_diffusion

    return updated_concentration


def update_right_side_concentration(
        cell_concentration,
        upper_cell_concentration,
        lower_cell_concentration,
        left_cell_concentration,
        diffusion_rate,
        time_step):
    """
        Update the concentration of a species in cells located along the right side of a 2D compartment
        (excluding the corners) based on diffusion from neighboring cells for each individual in population.

        Parameters:
        - cell_concentration (2d array): Array of concentrations for cells in the leftmost column (excluding corners).
        - upper_cell_concentration (2d array): Array of concentrations for cells directly above the current cells.
        - lower_cell_concentration (2d array): Array of concentrations for cells directly below the current cells.
        - left_cell_concentration (2d array): Array of concentrations for cells directly to the left of the current cells.
        - diffusion_rate (float): Rate at which the species diffuses between cells.
        - time_step (float): Discrete time step for the calculation.

        Returns:
        - numpy.ndarray: Updated concentrations of the species in the current cells.
        """
    upper_cell_in = time_step * upper_cell_concentration * diffusion_rate
    lower_cell_in = time_step * lower_cell_concentration * diffusion_rate
    right_cell_in = time_step * left_cell_concentration * diffusion_rate

    in_diffusion = upper_cell_in + lower_cell_in + right_cell_in
    out_diffusion = time_step * cell_concentration.T * diffusion_rate * 3

    updated_concentration = cell_concentration + in_diffusion - out_diffusion

    return updated_concentration


def update_central_concentration_middle(
        cell_concentration,
        upper_cell_concentration,
        lower_cell_concentration,
        right_cell_concentration,
        left_cell_concentration,
        diffusion_rate,
        time_step):
    """
    Update the concentration of species in multiple interior cells of a 2D compartment based on diffusion
    from neighboring cells for each individual in population.

    Parameters:
    - cell_concentration (2d array): Array of concentrations for multiple interior cells.
    - upper_cell_concentration (2d array): Array of concentrations for the cells directly above the current cells.
    - lower_cell_concentration (2d array): Array of concentrations for the cells directly below the current cells.
    - right_cell_concentration (2d array): Array of concentrations for the cells directly to the right of the current cells.
    - left_cell_concentration (2d array): Array of concentrations for the cells directly to the left of the current cells.
    - diffusion_rate (float): Rate at which the species diffuses between cells.
    - time_step (float): Discrete time step for the calculation.

    Returns:
    - ndarray: Updated concentrations of the species in the current interior cells.
    """
    upper_cell_in = time_step * upper_cell_concentration * diffusion_rate
    lower_cell_in = time_step * lower_cell_concentration * diffusion_rate
    right_cell_in = time_step * right_cell_concentration * diffusion_rate
    left_cell_in = time_step * left_cell_concentration * diffusion_rate

    in_diffusion = upper_cell_in + lower_cell_in + right_cell_in + left_cell_in
    out_diffusion = time_step * cell_concentration * diffusion_rate * 4

    updated_concentration = cell_concentration + in_diffusion - out_diffusion

    return updated_concentration


def update_central_concentration_upper(
        cell_concentration,
        lower_cell_concentration,
        right_cell_concentration,
        left_cell_concentration,
        diffusion_rate,
        time_step):
    """
    Update the concentration of a species in a cell located in the upper interior of a 2D compartment
    based on diffusion from neighboring cells for each individual in population.

    Parameters:
    - cell_concentration (1d array): Concentration of the species in the current upper interior cell.
    - lower_cell_concentration (1d array): Concentration of the species in the cell directly below the current cell.
    - right_cell_concentration (1d array): Concentration of the species in the cell directly to the right of the current cell.
    - left_cell_concentration (1d array): Concentration of the species in the cell directly to the left of the current cell.
    - diffusion_rate (float): Rate at which the species diffuses between cells.
    - time_step (float): Discrete time step for the calculation.

    Returns:
    - 1d array: Updated concentration of the species in the current upper interior cell.
    """
    lower_cell_in = time_step * lower_cell_concentration * diffusion_rate
    right_cell_in = time_step * right_cell_concentration * diffusion_rate
    left_cell_in = time_step * left_cell_concentration * diffusion_rate

    in_diffusion = lower_cell_in + right_cell_in + left_cell_in
    out_diffusion = time_step * cell_concentration * diffusion_rate * 3

    updated_concentration = cell_concentration + in_diffusion - out_diffusion

    return updated_concentration


def update_central_concentration_lower(
        cell_concentration,
        upper_cell_concentration,
        right_cell_concentration,
        left_cell_concentration,
        diffusion_rate,
        time_step):
    """
    Update the concentration of a species in a cell located in the lower interior of a 2D compartment
    based on diffusion from neighboring cells.

    Parameters:
    - cell_concentration (1d array): Concentration of the species in the current lower interior cell.
    - upper_cell_concentration (1d array): Concentration of the species in the cell directly above the current cell.
    - right_cell_concentration (1d array): Concentration of the species in the cell directly to the right of the current cell.
    - left_cell_concentration (1d array): Concentration of the species in the cell directly to the left of the current cell.
    - diffusion_rate (float): Rate at which the species diffuses between cells.
    - time_step (float): Discrete time step for the calculation.

    Returns:
    - 1d array: Updated concentration of the species in the current lower interior cell.
    """
    upper_cell_in = time_step * upper_cell_concentration * diffusion_rate
    right_cell_in = time_step * right_cell_concentration * diffusion_rate
    left_cell_in = time_step * left_cell_concentration * diffusion_rate

    in_diffusion = upper_cell_in + right_cell_in + left_cell_in
    out_diffusion = time_step * cell_concentration * diffusion_rate * 3

    updated_concentration = cell_concentration + in_diffusion - out_diffusion

    return updated_concentration



def apply_component_production(initial_concentration, production_pattern, production_rate, time_step):
    """
    Update the concentration of a species in each cell of a compartment.

    Parameters:
    - initial_concentration(1d array): Array of initial concentrations for each cell.
    - production_pattern(1d array): Array indicating which cells can produce the species.
    - production_rate(float): Rate at which the species are produced.
    - time_step(float): Discrete time step for the calculation.

    Returns:
    - Updated concentration array.
    """
    updated_concentration = np.maximum(initial_concentration + (production_pattern * production_rate * time_step), 0)

    return updated_concentration


def apply_component_degradation(initial_concentration, degradation_rate, time_step):
    """
    Apply degradation to the concentration of a species over time.

    Parameters:
    - initial_concentration (1d array): Array of initial concentrations for each cell.
    - degradation_rate (float): Rate at which the species degrades.
    - time_step (float): Discrete time step for the calculation.

    Returns:
    - Updated concentrations after applying degradation.
    """
    updated_concentration = np.maximum(initial_concentration - (initial_concentration * degradation_rate * time_step), 0)

    return updated_concentration


def apply_species_collision(species1, species2, complex_, collision_rate, time_step):
    """
    Apply the effect of species collision to form a complex and update the concentrations of the species.

    Parameters:
    - species1 (1d array): Array of concentrations of the first species.
    - species2 (1d array): Array of concentrations of the second species.
    - complex_ (1d array): Array of current concentrations of the complex.
    - collision_rate (float): Rate at which collisions occur between the two species.
    - time_step (float): Discrete time step  for the calculation.

    Returns:
    - tuple of numpy.ndarray: Updated concentrations of both species and the total amount of complex formed.
    """
    collision_effect = collision_rate * time_step
    complex_formed = np.minimum(species1 * collision_effect, species2 * collision_effect)
    complex_formed = np.maximum(complex_formed, 0)

    updated_species1 = np.maximum(species1 - complex_formed, 0)
    updated_species2 = np.maximum(species2 - complex_formed, 0)

    updated_complex = complex_ + complex_formed

    return updated_species1, updated_species2, updated_complex



def apply_complex_dissociation(species1, species2, complex_, dissociation_rate, time_step):
    """
    Apply the effect of complex dissociation to update the concentrations of the two species and the complex of them.

    Parameters:
    - species1 (1d array): Array of concentrations of the first species.
    - species2 (1d array): Array of concentrations of the second species.
    - complex_ (1d array): Array of current concentrations of the complex.
    - dissociation_rate (float): Rate at which the complex dissociates into the two species.
    - time_step (float): Discrete time step for the calculation.

    Returns:
    - tuple of numpy.ndarray: Updated concentrations of both species and the remaining amount of the complex.
    """
    dissociation_effect = dissociation_rate * time_step
    dissociated_amount = complex_ * dissociation_effect
    dissociated_amount = np.maximum(dissociated_amount, 0)

    updated_complex = np.maximum(complex_ - dissociated_amount, 0)
    updated_species1 = np.maximum(species1 + dissociated_amount, 0)
    updated_species2 = np.maximum(species2 + dissociated_amount, 0)

    return updated_species1, updated_species2, updated_complex



In [116]:

def run_simulation_with_timing():
    try:
        com_size = [10, 50, 100, 200, 500, 1000]
        com_time = []
        for c in com_size:
            tic = time.time()
            pop = np.zeros((7, c, c))
            pop[1, :, 0] = 10
            pop[3, :, -1] = 10

            pop[-1, 0, :3] = [.09, .007, 1.1]
            pop[-1, 2, :3] = [0.09, 0.006, 1.2]
            pop[-1, -1, :5] = [2, 1, 500, 5, .01]
            pop[-2, 0, 0:2] = [0, 2]
            pop[-2, 1, 0:4] = [6, .01, 0.001, 1.3]

            result = population_simulation(pop)
            toc = time.time()
            d = toc - tic
            com_time.append(d)
        
        max_s = [100, 500, 1000, 10000, 50000, 100000]
        dts = [.1, .02, .01, .001, 0.0002, 0.0001]
        sim_time = []
        for i in range(len(max_s)):
            tic = time.time()
            pop = np.zeros((7, 50, 50))
            pop[1, :, 0] = 10
            pop[3, :, -1] = 10

            pop[-1, 0, :3] = [.09, .007, 1.1]
            pop[-1, 2, :3] = [0.09, 0.006, 1.2]
            pop[-1, -1, :5] = [2, 1, max_s[i], 10, dts[i]]
            pop[-2, 0, 0:2] = [0, 2]
            pop[-2, 1, 0:4] = [6, .01, 0.001, 1.3]

            result = population_simulation(pop)
            toc = time.time()
            d = toc - tic
            sim_time.append(d)
        
        print("Compartment size times: ", com_time)
        print("Simulation time for different epochs and time steps: ", sim_time)
    
    except Exception as e:
        print(f"An error occurred: {e}")

run_simulation_with_timing()


Compartment size times:  [0.7420475482940674, 3.518643617630005, 7.3738672733306885, 18.19509768486023, 64.69217228889465, 231.44163966178894]
Simulation time for different epochs and time steps:  [0.6864268779754639, 3.468745470046997, 6.919107675552368, 69.68381881713867, 351.5794668197632, 711.421303987503]


In [117]:
times = []
pop_size = [20, 50, 100, 200, 500]
for p in pop_size:
    tic = time.time()
    for i in range(p):
        pop = np.zeros((7, 100, 100))
        pop[1, :, 0] = 10
        pop[3, :, -1] = 10

        pop[-1, 0, :3] = [.09, .007, 1.1]
        pop[-1, 2, :3] = [0.09, 0.006, 1.2]
        pop[-1, -1, :5] = [2, 1, 500, 5, .01]
        pop[-2, 0, 0:2] = [0, 2]
        pop[-2, 1, 0:4] = [6, .01, 0.001, 1.3]
        result = population_simulation(pop)
    toc = time.time()
    d = toc - tic
    times.append(d)

print(times)      

[150.99141120910645, 378.38582491874695, 752.4636449813843, 1492.9554994106293, 3762.6383543014526]


In [120]:
import json

# Define the benchmarking data
benchmarking_data = {
    "numba_com_size": {
        10: 10.832071781158447,
        50: 0.11558675765991211,
        100: 0.36243414878845215,
        200: 2.0385539531707764,
        500: 16.384703159332275,
        1000: 107.51107883453369,
        "max-epoch": 500
    },
    "numba_sim_epochs": {
        100: 0.0251920223236084,
        500: 0.11624932289123535,
        1000: 0.2307727336883545,
        10000: 2.3095428943634033,
        50000: 11.656088590621948,
        100000: 23.539443016052246,
        "com-size": (50, 50),
    },
    "numba_pop_size": {
        20: 18.735677242279053,
        50: 18.983787536621094,
        100: 38.32541823387146,
        200: 76.57288527488708,
        500: 191.06534576416016,
        "com-size": (100, 100),
        "max-epoch": 500,
        "population-size: 500; compartment-size:(30, 30)": 40.275675773620605
    },
    "com_size": {
        10: 0.7420475482940674,
        50: 3.518643617630005,
        100: 7.3738672733306885,
        200: 18.19509768486023,
        500: 64.69217228889465,
        1000: 231.44163966178894,
        "max-epoch": 500
    },
    "sim_epochs": {
        100: 0.6864268779754639,
        500: 3.468745470046997,
        1000: 6.919107675552368,
        10000: 69.68381881713867,
        50000: 351.5794668197632,
        100000: 711.421303987503,
        "com-size": (100, 100),
        "max-epoch": 500
    },
    "pop_size": {
        20: 150.99141120910645,
        50: 378.38582491874695,
        100: 752.4636449813843,
        200: 1492.9554994106293,
        500: 3762.6383543014526
    }
}

# Define the guide
guide = {
    "description": "This file contains benchmarking data for simulation algorithms tested with and without Numba optimization.",
    "variables": {
        "compartment_size": "The size of the 2D squared compartment where species spread out. Examples include (20, 20), (50, 50), etc.",
        "simulation_epochs": "Number of simulation epochs, indicating the duration of the simulation.",
        "population_size": "The size of the population being simulated, impacting the complexity and runtime of the simulation."
    },
    "note": "The data includes performance metrics for different configurations, comparing execution times with and without Numba optimizations."
}

# Save data to a JSON file
with open('benchmarking_data.json', 'w') as f:
    json.dump(benchmarking_data, f, indent=4)
    json.dump({"guide": guide}, f, indent=4)

print("Benchmarking data and guide successfully saved to 'benchmarking_data.json'.")



Benchmarking data and guide successfully saved to 'benchmarking_data.json'.
